In [1]:
import numpy as np
import copy
import glob
from matplotlib.pyplot import imread
from scipy.spatial.distance import cdist

In [2]:
# Parameters
nrun = 10 # number of classification runs
fname_label = 'class_labels.txt' # where class labels are stored for each run

In [3]:
def classification_run(folder,f_cost):
    # Compute error rate for one run of few-shot classification

    # get file names
    with open(folder+'/'+fname_label) as f:
        content = f.read().splitlines()
    pairs = [line.split() for line in content]
    test_files  = [pair[0] for pair in pairs]
    train_files = [pair[1] for pair in pairs]
    answers_files = copy.copy(train_files)
    test_files.sort()
    train_files.sort()
    ntrain = len(train_files)
    ntest = len(test_files)

    # load the images (and, if needed, extract features)
    train_items = [LoadImages(f, 'train') for f in train_files]
    test_items  = [LoadImages(test_files, 'test')]

    # create a numpy array for final results
    final = np.zeros((ntest))
    for i in range(ntest):
        # numpy array for temporary results
        results = np.zeros((len(train_items[0]), ntrain))
        for c in range(ntrain):
            # match each test item against all the training items.
            for d, val in enumerate(train_items[c]):
                cost = f_cost(test_items[0][i],val)
                results[d][c] = cost
        # find minimum value from temporary results
        results = np.amin(results, axis = 0)
        # find the minimum index from temporary results and assign as the final result.
        final[i] = np.argmin(results, axis = 0)

    # compute the error rate
    correct = 0.0
    for i in range(ntest):
        if train_files[int(final[i])] == answers_files[i]:
            correct += 1.0
    pcorrect = 100 * correct / ntest
    perror = 100 - pcorrect
    return perror

In [4]:
def ModHausdorffDistance(itemA,itemB):
    # Modified Hausdorff Distance
    #
    # Input
    #  itemA : [n x 2] coordinates of "inked" pixels
    #  itemB : [m x 2] coordinates of "inked" pixels
    #
    #  M.-P. Dubuisson, A. K. Jain (1994). A modified hausdorff distance for object matching.
    #  International Conference on Pattern Recognition, pp. 566-568.
    #
    D = cdist(itemA,itemB)
    mindist_A = D.min(axis=1)
    mindist_B = D.min(axis=0)
    mean_A = np.mean(mindist_A)
    mean_B = np.mean(mindist_B)
    return max(mean_A,mean_B)

In [5]:
def LoadImages(fn, type):
    paths = None
    if type == 'train':
        paths = glob.glob(fn + "/*.png")
    elif type == 'test':
        paths = fn
    else:
        assert False

    return LoadImgAsPoints(paths)


def LoadImgAsPoints(paths):
    # Load image file and return coordinates of 'inked' pixels in the binary image
    #
    # Output:
    #  return a list of D : [n x 2] rows are coordinates
    results = []

    for i in paths:
        I = imread(i)
        I = np.array(I,dtype=bool)
        I = np.logical_not(I)
        (row,col) = I.nonzero()
        D = np.array([row,col])
        D = np.transpose(D)
        D = D.astype(float)
        n = D.shape[0]
        mean = np.mean(D,axis=0)
        for i in range(n):
            D[i,:] = D[i,:] - mean
        results.append(D)

    return results

In [ ]:
    #   M.-P. Dubuisson, A. K. Jain (1994). A modified hausdorff distance for object matching.
    #     International Conference on Pattern Recognition, pp. 566-568.
    #
    print('few-shot classification demo with Modified Hausdorff Distance')
    perror = np.zeros(nrun)
    for r in range(1,nrun+1):
        rs = str(r)
        if len(rs)==1:
            rs = '0' + rs
        perror[r-1] = classification_run('data/run' + rs, ModHausdorffDistance)
        print (" run " + str(r) + " (error " + str(	perror[r-1] ) + "%)")
    total = np.mean(perror)
    print (" average error " + str(total) + "%")

few-shot classification demo with Modified Hausdorff Distance
 run 1 (error 10.0%)
 run 2 (error 0.0%)
 run 3 (error 20.0%)
 run 4 (error 10.0%)
 run 5 (error 15.0%)
 run 6 (error 20.0%)
